In [ ]:
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')

import torch
import torch.nn as nn
from torchsummary import summary
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import optim

In [ ]:
batch_size = 1000

In [ ]:
# 生成卷积操作时需要的两列数据的组合的列表
def generate_combination(N):
    """
    args:
        N: int, the number of rows of the matrix

    return:
        combination: list, the combination of two columns of the matrix
        combination_rev: list, the combination of two rows of the matrix, which is the reverse of combination
    """
    col = []
    col_rev = []
    for i in range(1,N):
        for j in range(0,i):
            col.append([i,j])
            col_rev.append([j,i])
    return col, col_rev
# 生成卷积操作时需要的两列数据的组合的列表
combination, combination_rev = generate_combination(9)

# 根据输入的矩阵和卷积操作的步长, 计算卷积操作的索引
def get_index_list(matrix, stride):
    """
    args:
        matrix: torch.tensor, the input matrix
        stride: int, the stride of the convolution operation
    
    return:
        index_list: list, the index of the convolution operation

    """
    W = matrix.shape[3]
    if W % stride == 0:
        index_list = list(np.arange(0, W+stride, stride))
    else:
        mod = W % stride
        index_list = list(np.arange(0, W+stride-mod, stride)) + [W]
    return index_list
# 根据输入的矩阵和卷积操作的步长, 计算卷积操作的索引
# Inception模块使用的卷积操作的步长为10
index_list = get_index_list(np.zeros((1,1,9,30)), 10)

In [ ]:
class Inception(nn.Module):
    """
    Inception, 用于提取时间序列的特征, 具体操作包括：

    1. kernel_size和stride均为d=10的特征提取层, 类似于卷积层，用于提取时间序列的特征. 具体包括:

        1. ts_corr4d: 过去 d 天 X 值构成的时序数列和 Y 值构成的时序数列的相关系数
        2. ts_cov4d: 过去 d 天 X 值构成的时序数列和 Y 值构成的时序数列的协方差
        3. ts_stddev4d: 过去 d 天 X 值构成的时序数列的标准差
        4. ts_zscore4d: 过去 d 天 X 值构成的时序数列的平均值除以标准差
        5. ts_return4d: (X - delay(X, d))/delay(X, d)-1, 其中delay(X, d)为 X 在 d 天前的取值
        6. ts_decaylinear4d: 过去 d 天 X 值构成的时序数列的加权平均值，权数为 d, d – 1, …, 1(权数之和应为 1，需进行归一化处理)，其中离现在越近的日子权数越大
        7. ts_mean4d: 过去 d 天 X 值构成的时序数列的平均值

        各操作得到的张量维数：
        1. 由于涉及两个变量的协方差, 因此ts_corr4d和ts_cov4d的输出为 N*1*36*3
        2. 其余操作均只涉及单变量的时序计算, 因此输出为 N*1*9*3

    2. 对第1步的输出进行Batch Normalization操作, 输出维数仍为 N*1*36*3 或 N*1*9*3

    3. 对于第2步得到的张量, kernel_size为3的池化层. 具体包括:
        1. max_pool: 过去 d 天 X 值构成的时序数列的最大值
        2. avg_pool: 过去 d 天 X 值构成的时序数列的平均值
        3. min_pool: 过去 d 天 X 值构成的时序数列的最小值

        以上三个操作的输出均为 N*1*117*1
    
    4. 对第3步的输出进行Batch Normalization操作, 输出维数仍为 N*1*117*1

    5. 将第2步和第4步的输出展平后进行拼接, 得到的张量维数为 N*(2*36*3+5*9*3+3*117) = N*702

    """

    def __init__(self, combination, combination_rev, index_list):
        """
        combination: 卷积操作时需要的两列数据的组合
        combination_rev: 卷积操作时需要的两列数据的组合, 与combination相反
        index_list: 卷积操作时需要的时间索引
        
        """

        super(Inception, self).__init__()
        # 卷积操作时需要的两列数据的组合
        self.combination = combination
        self.combination_rev = combination_rev

        # 卷积操作时需要的时间索引
        self.index_list = index_list
        self.d = len(index_list)-1

        # 卷积操作后的Batch Normalization层
        self.bc1 = nn.BatchNorm2d(1)
        self.bc2 = nn.BatchNorm2d(1)
        self.bc3 = nn.BatchNorm2d(1)
        self.bc4 = nn.BatchNorm2d(1)
        self.bc5 = nn.BatchNorm2d(1)
        self.bc6 = nn.BatchNorm2d(1)
        self.bc7 = nn.BatchNorm2d(1)

        # 池化层, 尺度为1*d
        self.max_pool = nn.MaxPool2d(kernel_size=(1, self.d))
        self.avg_pool = nn.AvgPool2d(kernel_size=(1, self.d))
        # 最小池化等价于相反数的最大池化, 后续会对结果取反
        self.min_pool = nn.MaxPool2d(kernel_size=(1, self.d))

        # 池化操作后的Batch Normalization层
        self.bc_pool1 = nn.BatchNorm2d(1)
        self.bc_pool2 = nn.BatchNorm2d(1)
        self.bc_pool3 = nn.BatchNorm2d(1)

    def forward(self, data):
        """
        data: 输入的数据, 维度为batch_size*1*9*30

        """
        # 本层的输入为batch_size*1*9*30, 在训练时不需要反向传播, 因此可以使用detach()函数
        data = data.detach().cpu().numpy()
        combination = self.combination
        combination_rev = self.combination_rev

        # 卷积操作
        conv1 = self.ts_corr4d(data, combination, combination_rev).to(torch.float)
        conv2 = self.ts_cov4d(data, combination, combination_rev).to(torch.float)
        conv3 = self.ts_stddev4d(data).to(torch.float)
        conv4 = self.ts_zcore4d(data).to(torch.float)
        conv5 = self.ts_return4d(data).to(torch.float)
        conv6 = self.ts_decaylinear4d(data).to(torch.float)
        conv7 = self.ts_mean4d(data).to(torch.float)

        # 卷积操作后的Batch Normalization
        batch1 = self.bc1(conv1)
        batch2 = self.bc2(conv2)
        batch3 = self.bc3(conv3)
        batch4 = self.bc4(conv4)
        batch5 = self.bc5(conv5)
        batch6 = self.bc6(conv6)
        batch7 = self.bc7(conv7)

        # 在 H 维度上进行特征拼接
        feature = torch.cat(
            [batch1, batch2, batch3, batch4, batch5, batch6, batch7], axis=2)  # N*1*(2*36+5*9)*3 = N*1*117*3

        # 同时将特征展平, 准备输入到全连接层
        feature_flatten = feature.flatten(start_dim=1)  # N*(117*3) = N*351

        # 对多通道特征进行池化操作, 每层池化后面都有Batch Normalization
        # 最大池化
        maxpool = self.max_pool(feature)  # N*1*117*1
        maxpool = self.bc_pool1(maxpool)
        # 平均池化
        avgpool = self.avg_pool(feature)  # N*1*117*1
        avgpool = self.bc_pool2(avgpool)
        # 最小池化
        # N*1*117*1, 最小池化等价于相反数的最大池化, 并对结果取反
        minpool = -self.min_pool(-1*feature)
        minpool = self.bc_pool3(minpool)
        # 特征拼接
        pool_cat = torch.cat([maxpool, avgpool, minpool],
                             axis=2)  # N*1*(3*117)*1 = N*1*351*1
        # 将池化层的特征展平
        pool_cat_flatten = pool_cat.flatten(start_dim=1)  # N*351

        # 拼接展平后的特征
        feature_final = torch.cat(
            [feature_flatten, pool_cat_flatten], axis=1)  # N*(351+351) = N*702
        return feature_final

    # 过去 d 天 X 值构成的时序数列和 Y 值构成的时序数列的相关系数
    def ts_corr4d(self, Matrix, combination, combination_rev):
        new_H = len(combination)
        index_list = self.index_list
        list = []  # 存放长度为len(index_list)-1的相关系数
        for i in range(len(index_list)-1):
            start_index = index_list[i]
            end_index = index_list[i+1]
            data = Matrix[:, :, combination, start_index:end_index]  # N*1*new_H*2*d
            data2 = Matrix[:, :, combination_rev,
                           start_index:end_index]  # N*1*new_H*2*d
            std1 = data.std(axis=4, keepdims=True)  # N*1*new_H*2*1, 在时序上求标准差
            std2 = data2.std(axis=4, keepdims=True)  # N*1*new_H*2*1, 在时序上求标准差
            std = (std1*std2).mean(axis=3, keepdims=True)  # N*1*new_H*1*1
            list.append(std)
        std = np.squeeze(np.array(list)).transpose(1, 2, 0).reshape(-1, 1, new_H,
                                                                    len(index_list)-1)+0.01  # N*1*new_H*len(index_list)-1 # 加上0.01, 防止除0
        # N*1*new_H*len(index_list)-1
        cov = self.ts_cov4d(Matrix, combination, combination_rev)
        corr = cov/std  # N*1*new_H*len(index_list)-1
        return corr

    # 过去 d 天 X 值构成的时序数列和 Y 值构成的时序数列的协方差
    def ts_cov4d(self, Matrix, combination, combination_rev):
        new_H = len(combination)
        index_list = self.index_list
        list = []  # 存放长度为len(index_list)-1的协方差
        for i in range(len(index_list)-1):
            start_index = index_list[i]
            end_index = index_list[i+1]
            data = Matrix[:, :, combination, start_index:end_index]  # N*1*new_H*2*d
            data2 = Matrix[:, :, combination_rev,
                           start_index:end_index]  # N*1*new_H*2*d
            mean1 = data.mean(axis=4, keepdims=True)  # N*1*new_H*2*1, 在时序上求均值
            mean2 = data2.mean(axis=4, keepdims=True)  # N*1*new_H*2*1, 在时序上求均值
            spread1 = data - mean1  # N*1*new_H*2*d, 在时序上求偏差
            spread2 = data2 - mean2  # N*1*new_H*2*d, 在时序上求偏差
            cov = ((spread1 * spread2).sum(axis=4, keepdims=True) /
                   (data.shape[4]-1)).mean(axis=3, keepdims=True)  # N*1*new_H*1*1
            list.append(cov)
        cov = np.squeeze(np.array(list)).transpose(
            1, 2, 0).reshape(-1, 1, new_H, len(index_list)-1)  # N*1*new_H*len(index_list)-1
        return torch.from_numpy(cov)

    # 过去 d 天 X 值构成的时序数列的标准差
    def ts_stddev4d(self, Matrix):
        # 只需要对单变量做卷积操作, 不需要将变量两两组合。因此输出的 H 可以保持和输入的 H 一致
        new_H = Matrix.shape[2]
        index_list = self.index_list
        list = []  # 存放长度为len(index_list)-1的标准差
        for i in range(len(index_list)-1):
            start_index = index_list[i]
            end_index = index_list[i+1]
            data = Matrix[:, :, :, start_index:end_index]  # N*1*H*d
            std = data.std(axis=3, keepdims=True)  # N*1*H*1
            list.append(std)
        std4d = np.squeeze(np.array(list)).transpose(
            1, 2, 0).reshape(-1, 1, new_H, len(index_list)-1)  # N*1*new_H*len(index_list)-1
        return torch.from_numpy(std4d)

    # 过去 d 天 X 值构成的时序数列的平均值除以标准差
    def ts_zcore4d(self, Matrix):
        # 只需要对单变量做卷积操作, 不需要将变量两两组合。因此输出的 H 可以保持和输入的 H 一致
        new_H = Matrix.shape[2]
        index_list = self.index_list
        list = []  # 存放长度为len(index_list)-1的zcore
        for i in range(len(index_list)-1):
            start_index = index_list[i]
            end_index = index_list[i+1]
            data = Matrix[:, :, :, start_index:end_index]  # N*1*H*d
            mean = data.mean(axis=3, keepdims=True)  # N*1*H*1
            std = data.std(axis=3, keepdims=True) + \
                0.01  # N*1*H*1, 加上0.01, 防止除以0
            list.append(mean/std)
        zscore = np.squeeze(np.array(list)).transpose(
            1, 2, 0).reshape(-1, 1, new_H, len(index_list)-1)  # N*1*new_H*len(index_list)-1
        return torch.from_numpy(zscore)

    # (X - delay(X, d))/delay(X, d)-1, 其中 delay(X, d)为 X 在 d 天前的取值
    def ts_return4d(self, Matrix):
        # 只需要对单变量做卷积操作, 不需要将变量两两组合。因此输出的 H 可以保持和输入的 H 一致
        new_H = Matrix.shape[2]
        index_list = self.index_list
        list = []  # 存放长度为len(index_list)-1的return
        for i in range(len(index_list)-1):
            start_index = index_list[i]
            end_index = index_list[i+1]
            data = Matrix[:, :, :, start_index:end_index]  # N*1*H*d
            # N*1*H*1, 在分母加上0.01, 防止除以0
            return_ = data[:, :, :, -1]/(data[:, :, :, 0]+0.01)-1
            list.append(return_)
        ts_return = np.squeeze(np.array(list)).transpose(
            1, 2, 0).reshape(-1, 1, new_H, len(index_list)-1)  # N*1*new_H*len(index_list)-1
        return torch.from_numpy(ts_return)

    # 过去 d 天 X 值构成的时序数列的加权平均值, 权数为 d, d – 1, …, 1(权数之和应为 1, 需进行归一化处理), 其中离现在越近的日子权数越大
    def ts_decaylinear4d(self, Matrix):
        new_H = Matrix.shape[2]
        index_list = self.index_list
        list = []  # 存放长度为len(index_list)-1的加权平均值
        for i in range(len(index_list)-1):
            start_index = index_list[i]
            end_index = index_list[i+1]
            range_ = end_index-start_index
            weight = np.arange(1, range_+1)
            weight = weight/weight.sum()  # 权重向量
            data = Matrix[:, :, :, start_index:end_index]  # N*1*H*d
            wd = (data*weight).sum(axis=3, keepdims=True)  # N*1*H*1
            list.append(wd)
        ts_decaylinear = np.squeeze(np.array(list)).transpose(
            1, 2, 0).reshape(-1, 1, new_H, len(index_list)-1)  # N*1*new_H*len(index_list)-1
        return torch.from_numpy(ts_decaylinear)

    # 过去 d 天 X 值构成的时序数列的平均值
    def ts_mean4d(self, Matrix):
        new_H = Matrix.shape[2]
        index_list = self.index_list
        list = []  # 存放长度为len(index_list)-1的平均值
        for i in range(len(index_list)-1):
            start_index = index_list[i]
            end_index = index_list[i+1]
            data = Matrix[:, :, :, start_index:end_index]  # N*1*H*d
            mean_ = data.mean(axis=3, keepdims=True)  # N*1*H*1
            list.append(mean_)
        ts_mean = np.squeeze(np.array(list)).transpose(
            1, 2, 0).reshape(-1, 1, new_H, len(index_list)-1)  # N*1*new_H*len(index_list)-1
        return torch.from_numpy(ts_mean)

In [ ]:
class AlphaNet(nn.Module):

    def __init__(self, combination, combination_rev, index_list, fc1_num, fc2_num, dropout_rate):
        super(AlphaNet, self).__init__()
        self.combination = combination
        self.combination_rev = combination_rev
        self.fc1_num = fc1_num
        self.fc2_num = fc2_num
        # 自定义的Inception模块
        self.Inception = Inception(combination, combination_rev, index_list)
        # 两个全连接层
        self.fc1 = nn.Linear(fc1_num, fc2_num) # 702 -> 30
        self.fc2 = nn.Linear(fc2_num, 1) # 30 -> 1
        # 激活函数
        self.relu = nn.ReLU()
        # dropout
        self.dropout = nn.Dropout(dropout_rate)
        # 初始化权重
        self._init_weights()

    def _init_weights(self):
        # 使用xavier的均匀分布对weights进行初始化
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        # 使用正态分布对bias进行初始化
        nn.init.normal_(self.fc1.bias, std=1e-6)
        nn.init.normal_(self.fc2.bias, std=1e-6)

    def forward(self, data):
        data = self.Inception(data)  # N*702
        data = self.fc1(data) # N*30
        data = self.relu(data)
        data = self.dropout(data)
        data = self.fc2(data) # N*1
        # 线性激活函数, 无需再进行激活
        data = data.to(torch.float)

        return data


In [ ]:
test = AlphaNet(combination, combination_rev, index_list, fc1_num=702, fc2_num=30, dropout_rate=0.5)

In [ ]:
summary(test, input_size=(1, 9, 30))

In [ ]:
# 读取数据
X_train = np.load('../data/X_train.npy')
y_train = np.load('../data/y_train.npy')
X_test = np.load('../data/X_test.npy')
y_test = np.load('../data/y_test.npy')
# 查看数据的大小
print("训练集特征维数: ", X_train.shape)
print("训练集标签维数: ", y_train.shape)
print("测试集特折维数: ", X_test.shape)
print("测试集标签维数: ", y_test.shape)

# 将数据转换为tensor
trainx = torch.from_numpy(np.array(X_train)).reshape(
    len(X_train), 1, 9, 30)  # 训练集的特征
trainy = torch.from_numpy(np.array(y_train)).reshape(
    len(y_train), 1)  # 训练集的标签
testx = torch.from_numpy(np.array(X_test)).reshape(
    len(X_test), 1, 9, 30)  # 测试集的特征
testy = torch.from_numpy(np.array(y_test)).reshape(
    len(y_test), 1)  # 测试集的标签

In [ ]:
class FactorData(Dataset):

    def __init__(self, train_x, train_y):
        self.len = len(train_x)
        self.x_data = train_x
        self.y_data = train_y

    def __getitem__(self, index):
        """
        指定读取数据的方式: 根据索引index返回dataset[index]

        """
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len


# 将数据载入到DataLoader中
train_data = FactorData(trainx, trainy)
train_loader = DataLoader(dataset=train_data,
                          batch_size=batch_size,
                          shuffle=False)  # 不打乱数据集
test_data = FactorData(testx, testy)
test_loader = DataLoader(dataset=test_data,
                         batch_size=batch_size,
                         shuffle=False)  # 不打乱数据集


In [ ]:
# 构建模型
alphanet = AlphaNet(combination=combination, combination_rev=combination_rev, index_list=index_list, fc1_num=702, fc2_num=30, dropout_rate=0.5)

In [ ]:
#weight decay: 对所有weight参数进行L2正则化
weight_list,bias_list = [],[]
for name,p in alphanet.named_parameters():
    if 'bias' in name:
        bias_list += [p]
    else:
        weight_list += [p]
optimizer = optim.SGD([{'params': weight_list, 'weight_decay':1e-5},
                       {'params': bias_list, 'weight_decay':0}],
                      lr = 1e-3,
                      momentum = 0.9)
criterion = nn.MSELoss()
#training
epoch_num = 1
loss_list = []
test_loss = []
for epoch in range(epoch_num ):
    for data,label in train_loader:
        # break
        #训练模式
        alphanet.train()#training pattern, grad required.
        out_put = alphanet(data.to(torch.float))
        loss = criterion(out_put,label.to(torch.float))
        loss_list.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() #renew the parameters
        
        #测试模式
        alphanet.eval()
        y_pred = alphanet(testx.to(torch.float))
        testloss = criterion(y_pred,testy)
        test_loss.append(testloss.item())
        #print("ok")
    
    #print("current epoch time:",epoch+1)
    #print("current loss of epoch ",epoch+1,":", train_loss.item())


In [ ]:
#Visualization
with plt.style.context(['ggplot']):
    fig, ax = plt.subplots(figsize = (10,4))
    x = list(np.arange(1,len(test_loss)+1))
    y = test_loss 
    ax.plot(x,y)
    ax.legend(title = "Loss on Training Set")
    ax.autoscale(tight = True)
    ax.set(xlabel = 'Batch combination')
    ax.set(ylabel = 'Loss')